In [ ]:
import os
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# --- 1. CONFIGURACIÓN Y DESCARGA ---
# Asegúrate de tener tu archivo 'kaggle.json' en el directorio actual antes de ejecutar esto.

print("Configurando API de Kaggle...")
# Movemos el archivo de credenciales al lugar correcto (solo necesario en Colab/Linux)
if not os.path.exists('/root/.kaggle'):
    os.makedirs('/root/.kaggle')
    os.system('cp kaggle.json /root/.kaggle/')
    os.system('chmod 600 /root/.kaggle/kaggle.json')

print("Descargando dataset PetFinder (esto puede tardar unos minutos)...")
# Descargamos la competición específica
# El comando '!' ejecuta comandos de terminal desde el notebook
!kaggle competitions download -c petfinder-adoption-prediction

print("Descomprimiendo archivos...")
# Descomprimimos el zip
with zipfile.ZipFile("petfinder-adoption-prediction.zip", 'r') as zip_ref:
    zip_ref.extractall("petfinder_data")

print("¡Descarga y extracción completada!")

# --- 2. CARGA DE DATOS (TABULAR + IMAGEN) ---

# Rutas a los datos descomprimidos
base_dir = 'petfinder_data'
csv_path = os.path.join(base_dir, 'train/train.csv')
images_dir = os.path.join(base_dir, 'train_images')

# A) Cargar Datos Tabulares
df = pd.read_csv(csv_path)

# --- 3. VISUALIZACIÓN MULTIMODAL ---
# Vamos a mostrar los datos de una mascota aleatoria para ver la "fusión"

# Elegimos una fila aleatoria
random_row = df.sample(1).iloc[0]
pet_id = random_row['PetID']

# En PetFinder, las fotos se llaman: PetID-1.jpg (la primera foto de esa mascota)
img_name = f"{pet_id}-1.jpg"
img_path = os.path.join(images_dir, img_name)

# Preparamos la visualización
plt.figure(figsize=(12, 6))

# Subgráfico 1: La Imagen
plt.subplot(1, 2, 1)
if os.path.exists(img_path):
    img = mpimg.imread(img_path)
    plt.imshow(img)
    plt.title("Input Visual (CNN)")
    plt.axis('off')
else:
    plt.text(0.5, 0.5, 'Imagen no encontrada', ha='center')

# Subgráfico 2: Los Datos Tabulares
plt.subplot(1, 2, 2)
plt.axis('off')
info_text = (
    f"--- DATOS TABULARES (Input Clásico) ---\n\n"
    f"Type: {'Perro' if random_row['Type']==1 else 'Gato'}\n"
    f"Age: {random_row['Age']} meses\n"
    f"Breed ID: {random_row['Breed1']}\n"
    f"Gender: {random_row['Gender']}\n"
    f"Color: {random_row['Color1']}\n"
    f"MaturitySize: {random_row['MaturitySize']}\n"
    f"Vaccinated: {random_row['Vaccinated']}\n\n"
    f"--- TARGET (Objetivo) ---\n"
    f"AdoptionSpeed: {random_row['AdoptionSpeed']} (0=Rápido, 4=Lento)"
)
plt.text(0.1, 0.4, info_text, fontsize=12, family='monospace')

plt.tight_layout()
plt.show()

# Mostrar las primeras filas del dataframe para verificar
print(f"Dataset cargado. Total de filas: {df.shape[0]}")
df.head(3)